In [ ]:
import os
import re
import sys
import glob
import pickle
import shelve
import numpy as np
from scipy.fft import fft, fftfreq
from scipy.signal import find_peaks
from scipy.optimize import curve_fit
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.transforms as mtransforms
from matplotlib.colors import LogNorm, FuncNorm
from matplotlib.gridspec import GridSpec
from matplotlib.ticker import FixedLocator, NullLocator, FixedFormatter
from matplotlib.patches import Polygon
import seaborn as sns

fontsize = 7
lw = 0.75
COL_WIDTH = {1: 88/25.4, 2: 180/25.4} # column width in inches
matplotlib.rc('font', **{'family': 'Arial', 'size': fontsize})
matplotlib.rc('axes', **{'linewidth': 0.75, 'labelsize': fontsize})
matplotlib.rc('xtick', **{'labelsize': fontsize})
matplotlib.rc('ytick', **{'labelsize': fontsize})
matplotlib.rc('xtick.major', **{'width': lw, 'size':3})
matplotlib.rc('ytick.major', **{'width': lw, 'size':3})
matplotlib.rc('ytick.minor', **{'width': lw, 'size':1.5})

%matplotlib inline

if '..' not in sys.path:
    sys.path.append('..')
from dlml.data import load_data_files, load_data_areas

In [ ]:
def make_axes(rows, cols, x_offset, y_offset, x_space, y_space, squeeze=True):
    w = (1 - np.sum(x_offset) - x_space * (cols - 1)) / cols
    h = (1 - np.sum(y_offset) - y_space * (rows - 1)) / rows
    
    ax = [[plt.axes([x_offset[0] + (w + x_space) * j,
                     y_offset[0] + (h + y_space) * i,
                     w, h]) for j in range(cols)] for i in range(rows-1, -1, -1)]
    
    for row in ax:
        for a in row:
            for side in 'right','top':
                a.spines[side].set_visible(False)

    if squeeze:
        if rows == 1 and cols == 1:
            return ax[0][0]
        if rows == 1:
            return ax[0]
        if cols == 1:
            return [a[0] for a in ax]
        
    return ax

In [ ]:
def plot_correlations(R, p, R_ctrl, p_ctrl, edges, idx, ax, sort_freq=1.0,
                      vmin=None, vmax=None, legend_bbox=[0.4, -0.05]):
    if p is not None:
        R = R.copy()
        R[p > 0.05] = np.nan
    if R_ctrl is not None and p_ctrl is not None:
        R_ctrl = R_ctrl.copy()
        R_ctrl[p_ctrl > 0.05] = np.nan
    rows, cols = ax.shape
    if rows != len(idx):
        raise Exception('Number of rows of ax does not match len(idx)')
    R_mean = [np.nanmean(R[jdx], axis=0) for jdx in idx]
    R_abs_mean = [np.mean(np.abs(r), axis=1) for r in R_mean]
    if R_ctrl is not None:
        R_ctrl_mean = [np.nanmean(R_ctrl[jdx], axis=0) for jdx in idx]
        R_ctrl_abs_mean = [np.mean(np.abs(r), axis=1) for r in R_ctrl_mean]
    else:
        R_ctrl_mean = [None for _ in range(rows)]
        R_ctrl_abs_mean = None
    if np.isscalar(sort_freq):
        sort_freq += np.zeros(rows)
    edge = np.array([np.abs(edges - freq).argmin() for freq in sort_freq])
    for i in range(rows):
        kdx = np.argsort(R_mean[i][edge[i],:])
        R_mean[i] = R_mean[i][:,kdx]
        if R_ctrl is not None:
            R_ctrl_mean[i] = R_ctrl_mean[i][:,kdx]

    make_symmetric = False
    if vmin is None:
        vmin = min([r.min() for r in R_mean])
        make_symmetric = True
    if vmax is None:
        vmax = max([r.max() for r in R_mean])
        if make_symmetric:
            if vmax > np.abs(vmin):
                vmin = -vmax
            else:
                vmax = -vmin
    print(f'Color bar bounds: ({vmin:.2f},{vmax:.2f}).')
    ticks = np.linspace(vmin, vmax, 7)
    ticklabels = [f'{tick:.2f}' for tick in ticks]

    cmap = plt.get_cmap('bwr')
    y = edges[:-1] + np.diff(edges) / 2
    dfs = []
    for i in range(rows):
        for j,R in enumerate((R_mean[i], R_ctrl_mean[i])):
            if R is not None:
                x = np.arange(R.shape[-1])
                im = ax[i][j].pcolormesh(x, y, R, vmin=vmin, vmax=vmax, shading='auto', cmap=cmap)
                ax[i][j].set_xticks(np.linspace(0, x[-1], 3, dtype=np.int32))
                X,Y = np.meshgrid(x,y)
                dfs.append(pd.DataFrame(data={'Filter': X.flatten(),
                                              'Frequency': Y.flatten(),
                                              'R': R.flatten()}))
        if cols > 1:
            cbar = plt.colorbar(im, fraction=0.1, shrink=1, aspect=20, label='Correlation',
                                orientation='vertical', ax=ax[i][1], ticks=ticks)
            cbar.ax.set_yticklabels(ticklabels, fontsize=fontsize-1)
            if R_ctrl_abs_mean is not None:
                ax[i][-1].plot(R_abs_mean[i], y, 'r', lw=1, label='train.')
                ax[i][-1].plot(R_ctrl_abs_mean[i], y, 'g--', lw=1, label='untrain.')
                ax[i][-1].plot(R_abs_mean[i] - R_ctrl_abs_mean[i], y, 'k', lw=1, label='diff.')
                ax[i][-1].legend(loc='lower left', bbox_to_anchor=legend_bbox,
                                 frameon=False, fontsize=fontsize-1)
                dfs.append(pd.DataFrame(data={'Frequency': y,
                                              'R_trained': R_abs_mean[i],
                                              'R_untrained': R_ctrl_abs_mean[i],
                                              'Diff': R_abs_mean[i] - R_ctrl_abs_mean[i]}))
    for i in range(rows):
        for j in range(cols):
            ax[i][j].set_ylim(edges[[0,-2]])
            ax[i][j].set_yscale('log')

    sns.despine()
    return vmin, vmax, dfs

In [ ]:
def plot_layer_output_hist(Y, group_index, N_bins, cols=8, w=2, h=1.5, cmap=None, ax=None, labels=None):
    N_trials, N_samples, N_filters = Y.shape
    N_groups = len(group_index)
    N = np.zeros((N_filters, N_groups, N_bins))
    edges = np.zeros((N_filters, N_groups, N_bins+1))
    for i in range(N_filters):
        for j,jdx in enumerate(group_index):
            N[i,j,:],edges[i,j,:] = np.histogram(Y[jdx, :, i], N_bins)

    if cmap is None:
        cmap = plt.get_cmap('tab10', N_groups)
    if ax is None:
        rows = N_filters // cols
        fig,ax = plt.subplots(rows, cols, figsize=(w*cols, h*rows))
    else:
        fig = None
        N_filters = ax.size
    ax = ax.flatten()
    for i in range(N_filters):
        for j in range(N_groups):
            de = np.diff(edges[i, j, :])[0]
            col = np.max([[0,0,0], cmap(j)[:3] - 1/3 * np.ones(3)], axis=0)
            ax[i].bar(edges[i, j, :-1], N[i, j, :], width=de*0.8, align='edge',
                     facecolor=cmap(j), edgecolor=col, linewidth=0.5, alpha=0.85)
        xlim = [edges[i, :, 2:-3].min(), edges[i, j, 2:-3].max()]
        ylim = ax[i].get_ylim()
        if labels is not None:
            ax[i].text(xlim[0] - 0.1 * np.diff(xlim), ylim[1],
                       labels[i], fontsize=fontsize-1, verticalalignment='top',
                       horizontalalignment='left')
        ax[i].set_xticklabels([])
        ax[i].set_yticks(ax[i].get_ylim())
        ax[i].set_yticklabels([])
        for side in 'right','top':
            ax[i].spines[side].set_visible(False)
    if fig is not None:
        fig.tight_layout()
    return fig,ax

In [ ]:
def make_dfs_dict(keys):
    return {key: pd.DataFrame() for key in keys}

def save_dfs_dict(dfs, xls_file, index=False):
    if not isinstance(index, list):
        index = [index for _ in range(len(dfs))]
    with pd.ExcelWriter(xls_file) as writer:
        for (name,df),idx in zip(dfs.items(), index):
            df.to_excel(writer, sheet_name='Panel_'+name, header=True, index=idx)

In [ ]:
X, y, Xf = {}, {}, {}
group_index, n_mom_groups = {}, {}

In [ ]:
var_name = 'Vd_bus3'
area_ID = 1
if area_ID == 1:
    subset = 8
elif area_ID == 2:
    subset = 2
else:
    raise Exception(f'Unknown area ID: {area_ID}.')
var_names, area_IDs = [var_name], [area_ID]
data_file = f'traces_hist_spectra_comp_grid_area_{area_ID}_{var_name}.npz'
force = False
set_name = 'training'
if not os.path.isfile(data_file) or force:

    data_dir = '../data/IEEE39/converted_from_PowerFactory/all_stoch_loads/' + \
        f'var_H_area_{area_ID}_comp_grid/coarse_H_comp{area_ID}1_0.1/diagonal'
    data_files = sorted(glob.glob(data_dir + os.path.sep + f'*_{set_name}_set.h5'))
    generators_areas_map = [['G02', 'G03', 'Comp11'],
                            ['G04', 'G05', 'G06', 'G07', 'Comp21'],
                            ['G08', 'G09', 'G10', 'Comp31'],
                            ['G01']]
    generators_Pnom = {'G01': 10e9, 'G02': 700e6, 'G03': 800e6, 'G04': 800e6, 'G05': 300e6,
                       'G06': 800e6, 'G07': 700e6, 'G08': 700e6, 'G09': 1000e6, 'G10': 1000e6,
                       'Comp11': 100e6, 'Comp21': 100e6, 'Comp31': 100e6}
    area_measure = 'momentum'
    ret = load_data_areas({set_name: data_files}, var_names,
                          [generators_areas_map[i-1] for i in area_IDs],
                          generators_Pnom,
                          area_measure,
                          trial_dur=60,
                          max_block_size=1000,
                          use_tf=False,
                          add_omega_ref=True,
                          use_fft=False)
    t = ret[0]
    X_raw = ret[1][set_name]
    y[set_name] = ret[2][set_name]
    group_index[set_name] = [np.where(y[set_name] == mom)[0] for mom in np.unique(y[set_name])]
    n_mom_groups[set_name] = len(group_index[set_name])
    X_mean, X_std = X_raw.mean(axis=(1,2)), X_raw.std(axis=(1,2))
    X[set_name] = (X_raw - X_mean) / X_std
    X[set_name] = X[set_name].squeeze()
    y[set_name] = y[set_name].squeeze()

    dt = np.diff(t[:2])[0]
    N_samples = t.size
    Xf[set_name] = fft(X[set_name])
    Xf[set_name] = 2.0 / N_samples * np.abs(Xf[set_name][:, :N_samples//2])
    F = fftfreq(N_samples, dt)[:N_samples//2]

    data_dir = '../data/IEEE39/converted_from_PowerFactory/all_stoch_loads/' + \
        f'var_H_area_{area_ID}_comp_grid/subset_{subset}_H_comp{area_ID}1_0.1'
    data_files = sorted(glob.glob(data_dir + os.path.sep + f'*_{set_name}_set.h5'))
    ret = load_data_areas({set_name: data_files}, var_names,
                          [generators_areas_map[i-1] for i in area_IDs],
                          generators_Pnom,
                          area_measure,
                          trial_dur=60,
                          max_block_size=1000,
                          use_tf=False,
                          add_omega_ref=True,
                          use_fft=False)    
    X_raw = ret[1][set_name]
    y['var_G2_G3'] = ret[2][set_name]
    group_index['var_G2_G3'] = [np.where(y['var_G2_G3'] == mom)[0] for mom in np.unique(y['var_G2_G3'])]
    n_mom_groups['var_G2_G3'] = len(group_index['var_G2_G3'])
    X['var_G2_G3'] = (X_raw - X_mean) / X_std
    X['var_G2_G3'] = X['var_G2_G3'].squeeze()
    y['var_G2_G3'] = y['var_G2_G3'].squeeze()
    Xf['var_G2_G3'] = fft(X['var_G2_G3'])
    Xf['var_G2_G3'] = 2.0 / N_samples * np.abs(Xf['var_G2_G3'][:, :N_samples//2])

    np.savez_compressed(data_file, t=t, X=X, y=y, Xf=Xf, F=F,
                        group_index=group_index, n_mom_groups=n_mom_groups)
else:
    data = np.load(data_file, allow_pickle=True)
    t = data['t']
    X = data['X'].item()
    y = data['y'].item()
    F = data['F']
    Xf = data['Xf'].item()
    group_index = data['group_index'].item()
    n_mom_groups = data['n_mom_groups'].item()
    
mom = np.array([y[set_name][idx].mean() for idx in group_index[set_name]])
std = np.array([X[set_name][idx,:].std() for idx in group_index[set_name]])

## Figure 2

In [ ]:
fig,ax = plt.subplot_mosaic(
    '''
    aaaaaaaa
    bbbbbccc
    ddddeeee
    ''',
    figsize=(COL_WIDTH[1], 4)
)
cmap = plt.get_cmap('Set1')
N_div_cmap = 10
div_cmap = plt.get_cmap('bwr', N_div_cmap)

###########################################
momentum = lambda H, S, fn: 2 * H@S / fn * 1e-3

N = 11, 11
h_G2_0, h_G3_0 = 4.33, 4.47
h_G2 = h_G2_0 + np.linspace(-1, 1, N[0])
h_G3 = h_G3_0 + np.linspace(-1, 1, N[1])
H_G2, H_G3 = np.meshgrid(*[h_G2, h_G3])

M = np.zeros(N)
S = np.array([700, 800])
fn = 60
for i in range(N[0]):
    for j in range(N[1]):
        H = np.array([H_G2[i,j], H_G3[i,j]])
        M[i,j] = momentum(H, S, fn)
        
gray_cmap = plt.get_cmap('gray')
cont = ax['a'].contourf(H_G2, H_G3, M, levels=100, cmap=gray_cmap, zorder=-1)
cbar = plt.colorbar(cont, ax=ax['a'])
white = [1,1,1]
magenta = [1,0,1]
green = [0,1,0]
yellow = [1,1,0]
blue = [0,.5,1]
red = [1,.333,.333]
orange = [1, .5, 0]
gray = [.6, .6, .6]
zord = 0
for i in range(2):
    for j in range(2):
        ax['a'].plot(H_G2[i,j], H_G3[i,j], 's', markersize=4, lw=1, color=div_cmap(i*2+j), zorder=zord)
        ax['a'].plot(H_G2[-1-i,-1-j], H_G3[-1-i,-1-j], 's', markersize=4, lw=1,
                     color=div_cmap(N_div_cmap-1-i*2-j), zorder=zord)
        zord += 2
for i in range(0, N[0], 2):
    ax['a'].plot(H_G2[i,i], H_G3[i,i], 'o', color=cmap(i//2), markersize=3, lw=1, zorder=zord)
    zord += 1
ax['a'].plot(H_G2[:2,:2].mean(), H_G3[:2,:2].mean(), 'x', color=magenta, markersize=4,
             markeredgewidth=1, zorder=zord)
zord += 1
ax['a'].plot(H_G2[-2:,-2:].mean(), H_G3[-2:,-2:].mean(), 'x', color=magenta, markersize=4,
             markeredgewidth=1, zorder=zord)
zord += 1
ax['a'].scatter(H_G2[::2, ::2], H_G3[::2, ::2], s=5, c='w', edgecolors='k', lw=0.5, marker='o', zorder=zord)
ax['a'].set_xlabel(r'$H_{G_2}$ [s]')
ax['a'].set_ylabel(r'$H_{G_3}$ [s]')
ax['a'].set_xlim([h_G2_0 - 1.1, h_G2_0 + 1.1])
ax['a'].set_ylim([h_G3_0 - 1.1, h_G3_0 + 1.1])
ax['a'].set_xticks([h_G2_0 - 1, h_G2_0, h_G2_0 + 1])
ax['a'].set_yticks([h_G3_0 - 1, h_G3_0, h_G3_0 + 1])
cbar.set_label(r'Momentum [GW$\cdot$s$^2$]')
cbar.set_ticks(np.r_[0.17 : 0.28 : 0.02])
###########################################

dfs = make_dfs_dict(('b','c','c-inset','d','e'))
tend = 10
jdx, = np.where(t < tend)
dfs['b']['Time'] = t[jdx]
dfs['d']['Frequency'] = F
dfs['e']['Frequency'] = F
for i,idx in enumerate(group_index[set_name]):
    n,edges = np.histogram(X[set_name][idx,:], bins=50, density=True)
    ax['b'].plot(t[jdx], X[set_name][idx[0]+1, jdx], lw=1, color=cmap(i))
    key = 'M={:.4f}'.format(np.unique(y[set_name])[i])
    dfs['b']['V_' + key] = X[set_name][idx[0]+1,jdx]
    ax['c'].plot(n, edges[1:], lw=1, color=cmap(i))
    dfs['c']['edges_' + key] = edges[1:]
    dfs['c']['distr_' + key] = n
    ax['d'].plot(F, 20 * np.log10(Xf[set_name][idx,:].mean(axis=0)), lw=1,
               color=cmap(i), label=r'{:.3f} GW$\cdot$s$^2$'.format(y[set_name][idx[0]+1]))
    dfs['d']['Power_' + key] = 20 * np.log10(Xf[set_name][idx,:].mean(axis=0))

for i,idx in enumerate(group_index['var_G2_G3']):
    if i < 4:
        ax['e'].plot(F, 20 * np.log10(Xf['var_G2_G3'][idx,:].mean(axis=0)), lw=1,
                   color=div_cmap(i))
    else:
        ax['e'].plot(F, 20 * np.log10(Xf['var_G2_G3'][idx,:].mean(axis=0)), lw=1,
                   color=div_cmap(N_div_cmap - (i - 3)))
    key = 'M={:.4f}'.format(np.unique(y['var_G2_G3'])[i])
    dfs['e']['Power_' + key] = 20 * np.log10(Xf['var_G2_G3'][idx,:].mean(axis=0))

for key in 'bc':
    ax[key].grid(which='major', axis='y', lw=0.5, ls=':', color=[.6,.6,.6])
for key in 'de':
    ax[key].grid(which='major', axis='x', lw=0.5, ls=':', color=[.6,.6,.6])
for a in ax.values():
    for side in 'right','top':
        a.spines[side].set_visible(False)
for key in 'bc':
    ax[key].set_ylim([-3.5, 3.5])
    ax[key].set_yticks(np.r_[-3 : 4 : 1.5])
for key in 'de':
    ax[key].set_ylim([-55, -10])
    ax[key].set_yticks(np.r_[-50 : -9 : 10])
    ax[key].set_xscale('log')
    ax[key].set_xlabel('Frequency [Hz]')
    ticks = np.array([0.1, 0.2, 0.5, 1, 2, 5, 10, 20])
    ax[key].set_xlim(ticks[[0,-1]] + np.array([0,1]))
    ax[key].xaxis.set_major_locator(FixedLocator(ticks))
    ax[key].xaxis.set_minor_locator(NullLocator())
    ax[key].xaxis.set_major_formatter(FixedFormatter([f'{tick:g}' for tick in ticks]))
ax['c'].set_yticklabels([])
ax['e'].set_yticklabels([])

ax['b'].set_xlabel('Time [s]')
ax['b'].set_ylabel('Norm. V')
ax['c'].set_xlabel('Distr.')
ax['d'].set_ylabel('Power [dB]')

ticks = np.r_[0 : 10.5 : 2]
ax['b'].set_xlim(ticks[[0,-1]])
ax['b'].xaxis.set_major_locator(FixedLocator(ticks))
ax['b'].xaxis.set_major_formatter(FixedFormatter([f'{tick:g}' for tick in ticks]))

ticks = np.r_[0 : 0.51 : 0.25]
ax['c'].set_xlim(ticks[[0,-1]])
ax['c'].xaxis.set_major_locator(FixedLocator(ticks))
ax['c'].xaxis.set_major_formatter(FixedFormatter([f'{tick:g}' for tick in ticks]))

inset = fig.add_axes([0.85, 0.62, 0.13, 0.09])
for i,(m,s) in enumerate(zip(mom,std)):
    inset.plot(m, s, 'o', ms=2.5, color=cmap(i))
dfs['c-inset'] = pd.DataFrame(data={'Momentum': mom, 'Stddev': std})
for side in 'right','top':
    inset.spines[side].set_visible(False)
xlim = [mom.min()*0.95, mom.max()*1.05]
ylim = [std.min()*0.95, std.max()*1.05]
inset.set_xlim(xlim)
inset.set_ylim(ylim)
xticks = [mom.min(), mom.max()]
yticks = [std.min(), std.max()]
inset.xaxis.set_major_locator(FixedLocator(xticks))
inset.xaxis.set_major_formatter(FixedFormatter([f'{tick:.2f}' for tick in xticks]))
inset.yaxis.set_major_locator(FixedLocator(yticks))
inset.yaxis.set_major_formatter(FixedFormatter([f'{tick:.2f}' for tick in yticks]))
    
trans = mtransforms.ScaledTranslation(-0.45, -0.05, fig.dpi_scale_trans)
for label in 'abd':
    ax[label].text(0.0, 1.0, label, transform=ax[label].transAxes + trans,
                   fontsize=fontsize, fontweight='bold', va='bottom')
trans = mtransforms.ScaledTranslation(-0.15, -0.05, fig.dpi_scale_trans)
for label in 'ce':
    ax[label].text(0.0, 1.0, label, transform=ax[label].transAxes + trans,
                   fontsize=fontsize, fontweight='bold', va='bottom')

fig.tight_layout(pad=0.2)
plt.savefig('traces_hist_spectra_comp_grid.pdf')
save_dfs_dict(dfs, 'figure_2.xlsx')

## Figure 3

In [ ]:
experiment_ID = '474d2016e33b441889ce8b17531487cb' # replaces '98475b819ecb4d569646d7e1467d7c9c'
# experiment_ID = 'd0e4cb94211c4190828fd8cd856cdd94' # replaces 'ed79ae2784274401a9dba5f5ccee98d8'
experiments_path = '../experiments/neural_network/'
history = pickle.load(open(os.path.join(experiments_path, experiment_ID, 'history.pkl'), 'rb'))
test_results = pickle.load(open(os.path.join(experiments_path, experiment_ID, 'test_results.pkl'), 'rb'))
N_bands = 60
filter_order = 6
if experiment_ID == '474d2016e33b441889ce8b17531487cb': # replaces '98475b819ecb4d569646d7e1467d7c9c'
    N_trials = 4000
elif experiment_ID == 'd0e4cb94211c4190828fd8cd856cdd94': # replaces 'ed79ae2784274401a9dba5f5ccee98d8'
    N_trials = 1000
else:
    raise Exception(f'Unknown number of trials for experiment {experiment_ID[:6]}')
correlations_file = f'correlations_{experiment_ID[:6]}_{N_bands}-bands_64-filters_' + \
    f'36-neurons_{N_trials}-trials_{filter_order}-butter_Vd_bus3_pool_1_3.npz'
correlations = np.load(os.path.join(experiments_path, experiment_ID, correlations_file))
for key in correlations.files:
    exec(f'{key} = correlations["{key}"]')
corr_idx = idx
corr_edges = edges

In [ ]:
key = 'var_G2_G3'

fig = plt.figure(figsize=(COL_WIDTH[2], 4))

offset = 0.07, 0.1
border = 0.01, 0.02
space = ((0.05, 0.1, 0.1), (0.075, 0.075, 0.13)), 0.17
rows = 2
h = (1 - offset[1] - border[1] - space[1] * (rows-1)) / rows
w_rel = [
    [3, 2, 3, 2],
    [2, 3, 3, 2]
]

w_rel_sum = np.sum(w_rel, axis=1)
cols = list(map(len, w_rel))
w_total = [1 - offset[0] - border[0] - np.sum(sp) for sp in space[0]]
w = []
for i in range(rows):
    w.append([])
    for j in range(cols[i]):
        w[-1].append(w_total[i] * w_rel[i][j] / w_rel_sum[i])

labels = ['abcd','efgh']
ax = {}
for i in range(rows):
    for j in range(cols[i]):
        ax[labels[i][j]] = fig.add_axes([offset[0] + np.sum(space[0][i][:j]) + np.sum(w[i][:j]),
                                         1 - border[1] - h * (i+1) - space[1] * i,
                                         w[i][j],
                                         h])

dfs = make_dfs_dict([key for key in ax])
# ############# Panel A #############
cmap = plt.get_cmap('tab10')
green, magenta = cmap(2), cmap(6)
jdx, = np.where(t < tend)
ax['a'].plot(t[jdx], X[key][:5, jdx].T, color=green, lw=0.5)
ax['a'].plot(t[jdx], X[key][-10:-5, jdx].T, color=magenta, lw=0.5)
dfs['a']['Time'] = t[jdx]
M = 'M={:.4f}'.format(np.unique(y[set_name])[0])
for i in range(5):
    dfs['a'][f'V_{M}_{i+1}'] = X[key][i, jdx]
M = 'M={:.4f}'.format(np.unique(y[set_name])[-1])
for i in range(5):
    dfs['a'][f'V_{M}_{i+1}'] = X[key][-1-i, jdx]

# ############# Panel B #############
idx = np.concatenate(group_index[key][:4])
n,edges = np.histogram(X[key][idx,:], bins=50, density=True)
ax['b'].plot(n, edges[1:], lw=1, color=green)
M = 'M={:.4f}'.format(np.unique(y[set_name])[0])
dfs['b']['edges_' + M] = edges[1:]
dfs['b']['distr_' + M] = n
m = Xf[key][idx,:].mean(axis=0)
ci = 1.96 * Xf[key][idx,:].std(axis=0) / np.sqrt(idx.size)
ax['e'].plot(20 * np.log10(m), F, color=green, lw=1)
dfs['e']['Frequency'] = F
dfs['e']['Power_' + M] = 20 * np.log10(m)
idx = np.concatenate(group_index[key][4:])
n,edges = np.histogram(X[key][idx,:], bins=50, density=True)
ax['b'].plot(n, edges[1:], lw=1, color=magenta)
M = 'M={:.4f}'.format(np.unique(y[set_name])[-1])
dfs['b'][f'edges_{M}'] = edges[1:]
dfs['b'][f'distr_{M}'] = n
m = Xf[key][idx,:].mean(axis=0)
ci = 1.96 * Xf[key][idx,:].std(axis=0) / np.sqrt(idx.size)
ax['e'].plot(20 * np.log10(m), F, color=magenta, lw=1)
dfs['e']['Power_' + M] = 20 * np.log10(m)
ax['b'].set_xlim([0, 0.5])
ticks = np.r_[0 : 0.51 : 0.25]
ax['b'].xaxis.set_major_locator(FixedLocator(ticks))
ax['b'].xaxis.set_major_formatter(FixedFormatter([f'{tick:g}' for tick in ticks]))
ax['b'].set_yticklabels([])

# ############# Panel C #############
ax['c'].plot(history['loss'], 'k', lw=1, label='Training')
ax['c'].plot(history['val_loss'], 'r', lw=1, label='Validation')
ax['c'].legend(loc='upper right', frameon=False, fontsize=fontsize)
dfs['c'] = pd.DataFrame(data={'Epoch': np.arange(len(history['loss']))+1,
                              'Loss': history['loss'],
                              'Validation_loss': history['val_loss']})

# ############# Panel D #############
target_values = np.unique(exact_momentum)
df = pd.DataFrame(data={'exact': exact_momentum, 'pred': np.concatenate(pred_momentum)})
df_ctrl = pd.DataFrame(data={'exact': exact_momentum, 'pred': np.concatenate(pred_momentum_ctrl)})
sns.violinplot(x='exact', y='pred', data=df, cut=0, inner='quartile',
               palette=[green, magenta], ax=ax['d'], linewidth=0.5)
ax['d'].xaxis.set_major_locator(FixedLocator([0, 1]))
ax['d'].xaxis.set_major_formatter(FixedFormatter([f'{tick:.2f}' for tick in target_values]))
ax['d'].yaxis.set_major_locator(FixedLocator(target_values))
ax['d'].yaxis.set_major_formatter(FixedFormatter([f'{tick:.2f}' for tick in target_values]))
dfs['d'] = df

# ############# Panels F, G ########
_,_,corr_dfs = plot_correlations(R, p, R_ctrl, p_ctrl, corr_edges,
                          [np.concatenate(corr_idx)], sort_freq=[1.1],
                          ax=np.array([[ax['f'], ax['g'], ax['h']]]))
ax['h'].plot(np.zeros(2), ax['h'].get_ylim(), ':', lw=1, color=[.6,.6,.6])
ax['f'].set_title('Trained network', fontsize=fontsize)
ax['g'].set_title('Untrained network', fontsize=fontsize)
for df,k in zip(corr_dfs,'FGH'):
    dfs[k] = df

for label in 'ab':
    ax[label].set_ylim([-3.5, 3.5])
    ax[label].set_yticks(np.r_[-3 : 3.1])

for label in 'abe':
    ax[label].grid(which='major', axis='y', lw=0.5, ls=':', color=[.6,.6,.6])

for a in ax.values():
    for side in 'right','top':
        a.spines[side].set_visible(False)

for label in 'efg':
    ax[label].set_yscale('log')
    ax[label].set_ylim([0.05, 20])

ax['e'].set_xlim([-10, -55])
ax['e'].set_xticks(np.r_[-50 : -9 : 10])

for label in 'efgh':
    ticks = np.array([0.1, 0.2, 0.5, 1, 2, 5, 10, 20])
    ax[label].set_ylim(ticks[[0,-1]])
    ax[label].yaxis.set_major_locator(FixedLocator(ticks))
    ax[label].yaxis.set_minor_locator(NullLocator())
    ax[label].yaxis.set_major_formatter(FixedFormatter([f'{tick:g}' for tick in ticks]))

ax['a'].set_xlabel('Time [s]')
ax['a'].set_ylabel('Norm. V')
ax['b'].set_xlabel('Distribution')
ax['c'].set_xlabel('Epoch')
ax['c'].set_ylabel('Loss')
ax['d'].set_xlabel(r'Exact M [GW$\cdot$s$^2$]')
ax['d'].set_ylabel(r'Predicted M [GW$\cdot$s$^2$]')
ax['e'].set_xlabel('Power [dB]')
ax['e'].set_ylabel('Frequency [Hz]')
ax['f'].set_xlabel('Filter #')
ax['g'].set_xlabel('Filter #')
ax['h'].set_xlabel('Correlation')
ax['f'].set_xticklabels([1, 32, 64])
ax['g'].set_xticklabels([1, 32, 64])

trans = mtransforms.ScaledTranslation(-0.2, -0.05, fig.dpi_scale_trans)
ax['b'].text(0.0, 1.0, 'b', transform=ax['b'].transAxes + trans, fontsize=fontsize,
             fontweight='bold', va='bottom')
trans = mtransforms.ScaledTranslation(-0.55, -0.05, fig.dpi_scale_trans)
ax['c'].text(0.0, 1.0, 'c', transform=ax['c'].transAxes + trans, fontsize=fontsize,
             fontweight='bold', va='bottom')
trans = mtransforms.ScaledTranslation(-0.4, -0.05, fig.dpi_scale_trans)
for label in 'adefgh':
    ax[label].text(0.0, 1.0, label, transform=ax[label].transAxes + trans,
                   fontsize=fontsize, fontweight='bold', va='bottom')

plt.savefig(f'low_high_momentum_{experiment_ID[:6]}.pdf')
save_dfs_dict(dfs, 'figure_3.xlsx')

### Figure 4

In [ ]:
data = np.load(os.path.join(experiments_path, experiment_ID, 'stopband_momentum_estimation.npz'))
for key in data.files:
    exec(f'{key} = data["{key}"]')
N_bands = len(bands)
Xfm = np.zeros((len(group_index), F.size))
Xfci = np.zeros((len(group_index), F.size))
for i,idx in enumerate(group_index):
    Xfm[i] = Xf[idx].mean(axis=0)
    Xfci[i] = 1.96 * Xf[idx].std(axis=0) / np.sqrt(idx.size)

In [ ]:
fig,ax = plt.subplot_mosaic(
    '''
    aaaa.
    bbbb.
    ''',
    figsize=(COL_WIDTH[1], 4)
)

dfs = make_dfs_dict('ab')
cmap2 = plt.get_cmap('Set1')
y_m = [y[idx].mean() for idx in group_index]
y_s = [y[idx].std() for idx in group_index]
y_pred_m = np.array([[pred[jdx].mean() for jdx in group_index] for pred in y_pred])
y_pred_s = np.array([[pred[jdx].std() for jdx in group_index] for pred in y_pred])
add_band = False
last_band = N_bands + 1 if add_band else N_bands + 1
dfs['a']['Exact'] = np.concatenate((y_m, y_s))
dfs['a'].index = 'Mean low momentum', 'Mean high momentum', 'Stddev low momentum', 'Stddev high momentum'
for i in range(last_band):
    if i == 0:
        lbl = 'Broadband'
    else:
        lbl = f'({bands[i-1][0]}-{bands[i-1][1]:g}) Hz'
    dfs['a'][lbl] = np.concatenate((y_pred_m[i,:], y_pred_s[i,:]))
    ax['a'].plot(y_m, y_pred_m[i], color=cmap2(i), lw=1, label=lbl)
    for j in range(len(group_index)):
        ax['a'].plot(y_m[j] + np.zeros(2),
                   y_pred_m[i,j] + y_pred_s[i,j] * np.array([-1,1]),
                   color=cmap2(i), lw=1)
        ax['a'].plot(y_m[j] + y_s[j] * np.array([-1,1]),
                   y_pred_m[i,j] + np.zeros(2),
                   color=cmap2(i), lw=1)
        ax['a'].plot(y_m[j], y_pred_m[i,j], 'o', color=cmap2(i),
                   markerfacecolor='w', markersize=4, markeredgewidth=1)
for side in 'right','top':
    ax['a'].spines[side].set_visible(False)
ax['a'].legend(loc='center left', bbox_to_anchor=[0.875, 0.5], frameon=False, fontsize=fontsize-1)
ax['a'].set_xlabel(r'Exact M [GW$\cdot$s$^2$]')
ax['a'].set_ylabel(r'Predicted M [GW$\cdot$s$^2$]')
ax['a'].set_xlim(y_m + np.diff(y_m) * np.array([-1/10, 1/5]))
ax['a'].set_ylim(y_m + np.diff(y_m) / 3 * np.array([-1,1]))
ax['a'].xaxis.set_major_locator(FixedLocator(y_m))
ax['a'].xaxis.set_major_formatter(FixedFormatter([f'{tick:.2f}' for tick in y_m]))
ax['a'].yaxis.set_major_locator(FixedLocator(y_m))
ax['a'].yaxis.set_major_formatter(FixedFormatter([f'{tick:.2f}' for tick in y_m]))

axr = ax['b'].twinx()
dfs['b']['Frequency'] = F
for i,(m,ci,col) in enumerate(zip(Xfm, Xfci, (green,magenta))):
    ax['b'].plot(F, 20*np.log10(m), color=col,
                 label=r'M = {:.2f} GW$\cdot$s$^2$'.format(y[group_index[i]].mean()))
    k = 'M={:.4f}'.format(y[group_index[i]].mean())
    dfs['b']['Power_' + k] = 20*np.log10(m)
ax['b'].legend(loc='lower left', frameon=False, fontsize=fontsize-1, bbox_to_anchor=[0.0, 0.13])
axr.plot(ax['b'].get_xlim(), scores[0] + np.zeros(2), '--', color=cmap2(0), lw=2)
for i,band in enumerate(bands):
    if i >= last_band-1:
        break
    axr.axvline(band[0], color=[.6,.6,.6], ls=':', lw=0.5)
    axr.plot(band, scores[i+1] + np.zeros(2), color=cmap2(i+1), lw=2)
ax['b'].set_xlabel('Frequency [Hz]')
ax['b'].set_ylabel('Power [dB]')
axr.set_ylabel(r'R$^2$ score')
ax['b'].set_xscale('log')

ticks = np.array([0.1, 0.2, 0.5, 1, 2, 5, 10, 20])
ax['b'].set_xlim(ticks[[0,-1]] + np.array([0,1]))
ax['b'].xaxis.set_major_locator(FixedLocator(ticks))
ax['b'].xaxis.set_minor_locator(NullLocator())
ax['b'].xaxis.set_major_formatter(FixedFormatter([f'{tick:g}' for tick in ticks]))
axr.set_ylim((-0.25, 1.05))
axr.set_yticks(np.r_[-0.2 : 1.05 : 0.2])

trans = mtransforms.ScaledTranslation(-0.45, -0.05, fig.dpi_scale_trans)
for label in 'ab':
    ax[label].text(0.0, 1.0, label, transform=ax[label].transAxes + trans,
                   fontsize=fontsize, fontweight='bold', va='bottom')

fig.tight_layout(pad=0.2)
if add_band:
    fig.savefig(f'stopband_momentum_estimation_{experiment_ID}_{last_band}.pdf')
else:
    fig.savefig(f'stopband_momentum_estimation_{experiment_ID[:6]}.pdf')
save_dfs_dict(dfs, 'figure_4.xlsx', index=[True,False])

### Figure 5

In [ ]:
experiment_IDs = '474d2016e33b441889ce8b17531487cb', 'c6f72abb5e364c4cb7770250e135bd73'
experiments_path = '../experiments/neural_network/'

data, correlations = {}, {}
N_bands = 60
filter_order = 6
for experiment_ID,N_trials in zip(experiment_IDs, (4000,4000)):
    key = experiment_ID[:6]
    tmp = np.load(os.path.join(experiments_path, experiment_ID, f'variable_inertia_{key}.npz'),
                  allow_pickle=True)
    data[key] = {}
    for fname in tmp.files:
        try:
            exec(f'data["{key}"]["{fname}"] = tmp["{fname}"].item()')
        except:
            exec(f'data["{key}"]["{fname}"] = tmp["{fname}"]')
    correlations_file = f'correlations_{key}_{N_bands}-bands_64-filters_' + \
        f'36-neurons_{N_trials}-trials_{filter_order}-butter_Vd_bus3_pool_1_3.npz'
    correlations[key] = np.load(os.path.join(experiments_path, experiment_ID, correlations_file))

In [ ]:
fig = plt.figure(figsize=(COL_WIDTH[2], 4))

cols = 2
offset = np.array([[0.075, 0.1], [0.14, 0.03]])
space = {
    'ab': 0.125,
    'bc': 0.05,
    'lr': 0.1,
    'de': 0.125,
    'ef': 0.035
}
width = {'a': 0.4}
height = {'a': 0.4}
width['b'] = (width['a'] - space['bc']) / 2
width['c'] = width['b']
height['b'] = 1 - np.sum(offset[:,1]) - height['a'] - space['ab']
height['c'] = height['b']
width['d'] = 1 - np.sum(offset[:,0]) - space['lr'] - width['a'] + 0.1
width['e'], width['f'] = width['d'] - 0.06, width['d']
height['d'] = (1 - np.sum(offset[:,1]) - space['de'] - space['ef']) / 2.5
height['e'] = (1 - np.sum(offset[:,1]) - space['de'] - space['ef'] - height['d']) / 2
height['f'] = height['e']

ax = {
    'a': plt.axes([offset[0,0],
                   offset[0,1] + space['ab'] + height['b'],
                   width['a'], height['a']]),
    'b': plt.axes([offset[0,0],
                   offset[0,1],
                   width['b'],
                   height['b']]),
    'c': plt.axes([offset[0,0] + space['bc'] + width['b'],
                   offset[0,1],
                   width['c'], height['c']]),
    'd': plt.axes([offset[0,0] + width['a'] + space['lr'],
                   offset[0,1] + 2*height['e'] + space['ef'] + space['de'],
                   width['d'], height['d']]),
    'e': plt.axes([offset[0,0] + width['a'] + space['lr'],
                   offset[0,1] + height['f'] + space['ef'],
                   width['e'], height['e']]),
    'f': plt.axes([offset[0,0] + width['a'] + space['lr'],
                   offset[0,1],
                   width['f'], height['f']]),
}

dfs = make_dfs_dict('abcd')

def cmap(i):
    p = sns.color_palette('colorblind')
    return [p[0], p[1], p[2], p[4]][i]

################### Panels A, B and C ###################
key = experiment_IDs[0][:6]
xticks = {'a': np.array([0.1, 0.2, 0.5, 1, 2, 5, 10, 20]),
          'b': np.array([0.4, 0.5, 0.7, 1, 1.5]),
          'c': np.array([8, 10, 12, 15])}
yticks = {'a': np.r_[-60 : -5 : 10],
          'b': np.r_[-30 : -9 : 5],
          'c': np.r_[-56 : -44 : 3]}
dfs['a']['Frequency'] = F
for a in 'abc':
    n = 0
    for i,(k,v) in enumerate(data[key]['Xf'].items()):
        for j in range(data[key]['n_mom_groups'][k]):
            lbl = r'{:.3f} GW$\cdot$s$^2$'.format(data[key]['ym'][k][j])
            if k == 'var_G2_G3':
                lbl += ' (GEN)'
            elif k == 'var_Comp11':
                lbl += ' (COMP)'
            m = v[data[key]['group_index'][k][j], :].mean(axis=0)
            s = v[data[key]['group_index'][k][j], :].std(axis=0)
            ci = 1.96 * s / np.sqrt(data[key]['group_index'][k][j].size)
            ax[a].fill_between(data[key]['F'],
                               20*np.log10(m + ci),
                               20*np.log10(m - ci),
                               color=cmap(n), facecolor=cmap(n), edgecolor=cmap(n),
                               alpha=0.5, label=lbl)
            
            n += 1
            M = 'M={:.4f}{}'.format(data[key]['ym'][k][j], ' ' + lbl.split(' ')[-1] if k == 'var_G2_G3' else '')
            dfs['a']['Mean_power_' + M] = m
            dfs['a']['CI_power_' + M] = ci
    ax[a].set_xscale('log')
    ax[a].set_xlabel('Frequency [Hz]')
    for side in 'right','top':
        ax[a].spines[side].set_visible(False)
    ax[a].set_xlim(xticks[a][[0,-1]] + np.array([0,1]))
    ax[a].xaxis.set_major_locator(FixedLocator(xticks[a]))
    ax[a].xaxis.set_minor_locator(NullLocator())
    ax[a].xaxis.set_major_formatter(FixedFormatter([f'{tick:g}' for tick in xticks[a]]))
    ax[a].yaxis.set_major_locator(FixedLocator(yticks[a]))
    ax[a].yaxis.set_minor_locator(NullLocator())
    ax[a].yaxis.set_major_formatter(FixedFormatter([f'{tick:g}' for tick in yticks[a]]))

ax['a'].legend(loc='lower left', frameon=False, fontsize=6)
ax['a'].set_ylabel('Power [dB]')
ax['b'].set_ylabel('Power [dB]')
ax['a'].set_ylim([-60, -10])
ax['b'].set_xlim([0.4, 1.5])
ax['c'].set_xlim([8, 15])
ax['b'].set_ylim([-31, -9])
ax['c'].set_ylim([-57, -46])

################### Panel D ###################
key = experiment_IDs[0][:6]
ax['d'].plot(data[key]['ym']['test'], data[key]['ym']['test'], 'k--', lw=2, markerfacecolor='w')
m = 'so'
markers = {ID[:6]: m[i] for i,ID in enumerate(experiment_IDs)}
lw, ms = 1.5, 6
for ID in experiment_IDs:
    n = 0
    key = ID[:6]
    ym = data[key]['ym']
    ym_pred = data[key]['ym_pred']
    ys_pred = data[key]['ys_pred']
    for cond in ym:
        for i in range(len(ym[cond])):
            ax['d'].plot(ym[cond][i] + np.zeros(2),
                         ym_pred[cond][i] + ys_pred[cond][i] * np.array([-1,1]),
                         color=cmap(n), linewidth=lw)
            ax['d'].plot(ym[cond][i], ym_pred[cond][i], markers[key], color=cmap(n), markersize=ms,
                     markerfacecolor='w', markeredgewidth=lw)
            n += 1
dfs['b'] = pd.DataFrame(data={'Exact_M': ym['test'] + ym['var_G2_G3'] + ym['var_Comp11'],
                        'Predicted_M_mean': ym_pred['test'] + ym_pred['var_G2_G3'] + ym_pred['var_Comp11'],
                        'Predicted_M_stddev': ys_pred['test'] + ys_pred['var_G2_G3'] + ys_pred['var_Comp11']},
                  index=['Low momentum', 'High momentum', 'Varying H_G2+H_G3', 'Varying H_Comp11'])
ax['d'].plot(1, 1, 'k'+m[0], markersize=ms, markerfacecolor='w',
             markeredgewidth=lw, label='without var. comp. in training')
ax['d'].plot(1, 1, 'k'+m[1], markersize=ms, markerfacecolor='w',
             markeredgewidth=lw, label='with var. comp. in training')
ax['d'].legend(loc='lower right', frameon=False, fontsize=fontsize-1, bbox_to_anchor=[1, -0.05])
ax['d'].set_xlabel(r'Exact M [GW$\cdot$s$^2$]')
ax['d'].set_ylabel(r'Predicted M [GW$\cdot$s$^2$]')
ym = ym['test']
ticks = [ym[0], 0.197, ym[1]]
ax['d'].set_xlim(ym + np.diff(ym) * np.array([-1/10, 1/10]))
ax['d'].set_ylim(ym + np.diff(ym) * np.array([-1/10, 1/10]))
ax['d'].xaxis.set_major_locator(FixedLocator(ticks))
ax['d'].xaxis.set_major_formatter(FixedFormatter([f'{tick:.3f}' for tick in ticks]))
ax['d'].yaxis.set_major_locator(FixedLocator(ticks))
ax['d'].yaxis.set_major_formatter(FixedFormatter([f'{tick:.3f}' for tick in ticks]))

################### Panels E and F ###################
key = experiment_IDs[1][:6]
_,_,df = plot_correlations(correlations[key]['R'],
                                 correlations[key]['p'],
                                 None,
                                 None,
                                 correlations[key]['edges'],
                                 [np.concatenate(correlations[key]['idx'])],
                                 sort_freq=[1.5],
                                 ax=np.array([[ax['e']]]))
dfs['c'] = df[0]
_,_,df = plot_correlations(correlations[key]['R'],
                                 correlations[key]['p'],
                                 None,
                                 None,
                                 correlations[key]['edges'],
                                 [np.concatenate(correlations[key]['idx'])],
                                 sort_freq=[10],
                                 ax=np.array([[ax['f'], ax['f']]]))
dfs['d'] = df[0]
for lbl in 'ef':
    ax[lbl].set_ylim(xticks['a'][[0,-1]] + np.array([0,1]))
    ax[lbl].yaxis.set_major_locator(FixedLocator(xticks['a']))
    ax[lbl].yaxis.set_minor_locator(NullLocator())
    ax[lbl].yaxis.set_major_formatter(FixedFormatter([f'{tick:g}' for tick in xticks['a']]))
    ax[lbl].set_ylabel('Frequency [Hz]')
    ax[lbl].set_xticks([1, 32, 64])
ax['e'].set_xticklabels([])
ax['f'].set_xlabel('Filter #')

for lbl in ax:
    for side in 'right','top':
        ax[lbl].spines[side].set_visible(False)

trans = mtransforms.ScaledTranslation(-0.4, -0.04, fig.dpi_scale_trans)
ax['a'].text(0.0, 1.0, 'a', transform=ax['a'].transAxes + trans,
             fontsize=fontsize, fontweight='bold', va='bottom')
trans = mtransforms.ScaledTranslation(-0.45, -0.04, fig.dpi_scale_trans)
for key,lbl in zip('def', 'bcd'):
    ax[key].text(0.0, 1.0, lbl, transform=ax[key].transAxes + trans,
                 fontsize=fontsize, fontweight='bold', va='bottom')

pdf_file = f'variable_inertia_{experiment_IDs[0][:6]}_{experiment_IDs[1][:6]}.pdf'
fig.savefig(pdf_file)
save_dfs_dict(dfs, 'figure_5.xlsx', index=[i==1 for i in range(4)])

### Figure 6

In [ ]:
experiment_ID = 'f64bde90cab54d1ea770bb21f33c3ed1'
experiments_path = '../experiments/neural_network/'
history = pickle.load(open(os.path.join(experiments_path, experiment_ID, 'history.pkl'), 'rb'))
test_results = pickle.load(open(os.path.join(experiments_path, experiment_ID, 'test_results.pkl'), 'rb'))
print('MAPE on the test set: {:.2f}%.'.format(test_results['mape_prediction'][0]))

In [ ]:
fig,ax = plt.subplots(2, 1, figsize=(2.5,3))
dfs = {'top': pd.DataFrame(data={'Epoch': np.arange(len(history['loss']))+1,
                                 'Loss': history['loss'],
                                 'Validation_loss': history['val_loss']})}
ax[0].plot(history['loss'], 'k', lw=1, label='Training')
ax[0].plot(history['val_loss'], 'r', lw=1, label='Validation')
ax[0].set_xlabel('Epoch')
ax[0].set_ylabel('Loss')
ax[0].legend(loc='upper right', frameon=False)

y_test,y_pred = test_results['y_test'].squeeze(), test_results['y_prediction'].squeeze()
if experiment_ID[:6] == 'a40658':
    limits = [0.17, 0.3]
    ticks = np.r_[0.18 : 0.31 : 0.03]
elif experiment_ID[:6] == 'f64bde':
    limits = [0.17, 0.28]
    ticks = np.r_[0.18 : 0.31 : 0.03]
else:
    raise Exception('set limits and ticks')

dfs['bottom'] = pd.DataFrame(columns=('Exact_M', 'Predicted_M_mean', 'Predicted_M_stddev'))
ax[1].plot(limits, limits, '--', lw=1, color=[.6,.6,.6])
for i,y in enumerate(np.unique(y_test)):
    idx = y_test == y
    m = y_pred[idx].mean()
    s = y_pred[idx].std()
    ax[1].plot(y+np.zeros(2), m+s*np.array([-1,1]), 'k', lw=1)
    ax[1].plot(y, m, 'ko', markersize=4, markerfacecolor='w', markeredgewidth=1)
    dfs['bottom'].loc[i] = y, m, s
ax[1].set_xlabel(r'Exact M [GW$\cdot$s$^2$]')
ax[1].set_ylabel(r'Predicted M [GW$\cdot$s$^2$]')
ax[1].set_xticks(ticks)
ax[1].set_yticks(ticks)
ax[1].set_xlim(limits)
ax[1].set_ylim(limits)

for a in ax:
    for side in 'right','top':
        a.spines[side].set_visible(False)

fig.tight_layout(pad=0.1)
fig.savefig(f'training_{experiment_ID[:6]}.pdf')
save_dfs_dict(dfs, 'figure_6.xlsx')

### Figure 7

In [ ]:
experiment_IDs = 'a40658acee3c4e419c0ee34d0c59f4df', 'f64bde90cab54d1ea770bb21f33c3ed1'
experiments_path = '../experiments/neural_network/'
dbs = [shelve.open(os.path.join(experiments_path, ID, ID[:6]+'_Pfrac=0.1.out'), flag='r') 
       for ID in experiment_IDs]

In [ ]:
fig,ax = plt.subplot_mosaic(
    '''
    a
    b
    c
    d
    ''',
    figsize=(COL_WIDTH[1], 4)
)

ylim = [
    [0.17, 0.27],
    [0.17, 0.27],
    [0.20, 0.25],
    [0.20, 0.25]
]
yticks = [
    np.r_[0.18 : 0.27 : 0.04],
    np.r_[0.18 : 0.27 : 0.04],
    np.r_[0.20 : 0.26 : 0.025],
    np.r_[0.20 : 0.26 : 0.025],
]
nextch = lambda ch: chr(ord(ch) + 1)
ithch = lambda n,start='a': chr(ord(start)+n)

dfs = make_dfs_dict(ax.keys())

col = [.6+np.zeros(3), np.zeros(3)]
col = [[.2,.8,.4], np.zeros(3)]
magenta = [1,0,1]

comp = 'fixed', 'var'
for i,db in enumerate(dbs):
    for j,expt in enumerate(db['experiments'][:4]):
        J = ithch(j)
        time = expt['prediction_time']
        prediction = np.squeeze(expt['prediction'])
        exact = expt['exact']
        mean_prediction = expt['mean_prediction']
        dfs[J]['Time'] = time/60
        dfs[J][f'M_{comp[i]}_comp'] = prediction
        N_blocks = len(expt['H_values'])
        block_dur = np.ceil(expt['data_time'][-1]) / N_blocks
        area_measure = 'M'
        measure_units = r'GW$\cdot$s$^2$'
        for k in range(N_blocks):
            t0,t1 = block_dur*k, block_dur*(k+1)
            idx, = np.where((time >= t0) & (time < t1) & np.logical_not(np.isnan(prediction)))
            n,x = np.histogram(prediction[idx], bins=10, density=True)
            ax[J].plot(np.array([t0, t1])/60, mean_prediction[k] + np.zeros(2), color=col[i], lw=1)
            if i == 1:
                ax[J].plot(np.array([t0, t1])/60, exact[k] + np.zeros(2), '--', color=magenta, lw=1)
        ax[J].plot(time/60, prediction, color=col[i], lw=0.75)
        if i == 0:
            ax[J].grid(which='major', axis='y', lw=0.5, ls=':', color=[.6,.6,.6])
            ax[J].set_ylim(ylim[j])
            ax[J].set_xticks(np.r_[0 : 181 : 30])
            ax[J].set_yticks(yticks[j])
            ax[J].set_ylabel(f'{area_measure.capitalize()} [{measure_units}]')

ax['d'].set_xlabel('Time [min]')
for i in 'ab':
    ax[i].set_xticklabels([])
ax['c'].set_xticklabels([])
trans = mtransforms.ScaledTranslation(-0.5, -0.05, fig.dpi_scale_trans)
for lbl in 'abcd':
    ax[lbl].text(0.0, 1.0, lbl, transform=ax[lbl].transAxes + trans,
                 fontsize=fontsize, fontweight='bold', va='bottom')

sns.despine()
fig.tight_layout(pad=0.2)
pdf_file = f'area_momentum_estimation_grid_{experiment_IDs[0][:6]}_{experiment_IDs[1][:6]}.pdf'
fig.savefig(pdf_file)
save_dfs_dict(dfs, 'figure_7.xlsx')

### Figure 8

In [ ]:
buses = 3,
N_buses = len(buses)
var_names = [f'Vd_bus{bus}' for bus in buses]
area_measure = 'momentum'
max_block_size = 100
cutoff = 0.1
filename = f'spectra_compensators_buses={"-".join(map(str,buses))}_cutoff={cutoff:.02f}_blocks={max_block_size}'
data = np.load(filename + '.npz')
for key in data.files:
    globals()[key] = data[key]
min_fft = Xfm.min(axis=(1,2))
max_fft = Xfm.max(axis=(1,2))
N_H = len(H_comp)
step = Xfm.shape[1] // H_comp.size

n = Xfm.shape[1]
peaks = np.zeros((n,2))
var_idx = 0
for i in range(n):
    x = 20*np.log10(Xfm[var_idx, i, F < 1.5])
    locs,_ = find_peaks(x, height=10, prominence=1, distance=5)
    peaks[i,:] = F[locs[1:3]]

In [ ]:
fig = plt.figure(figsize=(COL_WIDTH[1], 5))
x_offset = [0.175, 0.19]
y_offset = [0.075, 0.03]
x_space = 0.1
y_space = 0.05

H_comp_sub = [1.0, 3.0, 6.0]
rows = len(H_comp_sub)
height = [0.15, 0.15]
n = 2.2
w = 1 - np.sum(x_offset)
h = (1 - np.sum(height) - y_space*len(height) - np.sum(y_offset) - y_space * (rows - 1)) / rows

ax = [plt.axes([x_offset[0], 1 - y_offset[1] - height[0], w, height[0]])]
for i in range(rows-1, -1, -1):
    ax.append(plt.axes([x_offset[0],
                        y_offset[0] + height[1] + y_space*1.7 + (h + y_space/2) * i,
                        w, h]))
ax.append(plt.axes([x_offset[0], y_offset[0], w, height[1]]))

dfs = make_dfs_dict('abc')

xticks = np.array([0.1, 0.2, 0.5, 1, 2, 5, 10, 20])
var_idx = 0

reds = plt.get_cmap('Reds', N_H+4)
blues = plt.get_cmap('Blues', N_H+4)
dfs['a']['F'] = F
for i in range(1, N_H):
    lbl = 'Low momentum' if i == N_H - 1 else None
    ax[0].plot(F, 20*np.log10(Xfm[var_idx, i*step, :]), color=reds(i+2), lw=2, label=lbl)
    lbl = 'High momentum' if i == N_H - 1 else None
    ax[0].plot(F, 20*np.log10(Xfm[var_idx, step - 1 + i*step, :]), color=blues(i+2), lw=1, label=lbl)
    dfs['a'][f'Low_M_H_comp={H_comp[i]:.1f}'] = 20*np.log10(Xfm[var_idx, i*step, :])
    dfs['a'][f'High_M_H_comp={H_comp[i]:.1f}'] = 20*np.log10(Xfm[var_idx, step - 1 + i*step, :])
ax[0].set_xscale('log')
ax[0].set_yticks(np.r_[-20:25:10])
ax[0].set_ylabel('Power [dB]')
ax[0].grid(which='major', axis='both', ls=':', lw=0.5, color=[.6,.6,.6])
ax[0].set_xlim(xticks[[0,-1]] + np.array([0,1]))
ax[0].xaxis.set_major_locator(FixedLocator(xticks))
ax[0].xaxis.set_minor_locator(NullLocator())
ax[0].xaxis.set_major_formatter(FixedFormatter([f'{tick:g}' for tick in xticks]))
ax[0].legend(loc='lower left', frameon=False, fontsize=fontsize-1, bbox_to_anchor=[0,-0.05])
ax[0].arrow(17, 7, -10, 7, shape='left', width=0.2, head_width=2, head_length=1, fc='k', ec='k')
ax[0].text(10, 27, 'Increasing H comp', fontsize=fontsize-2, va='top', ha='center', rotation=-15)
_forward = lambda x: 20 * np.log10(x)
_inverse = lambda y: 10 ** (y/20)
cmap = plt.cm.inferno
cbar_ticks = np.r_[-20 : 35 : 10]
for i,H in enumerate(H_comp_sub):
    j = np.where(H_comp == H)[0][0]
    idx = IDX[j]
    norm = FuncNorm((_forward, _inverse), vmin=_inverse(cbar_ticks[0]), vmax=_inverse(cbar_ticks[-1]))
    X,Y = np.meshgrid(F, ym[idx])
    ylim = [Y.min(), Y.max()]
    ax[i+1].imshow(Xfm[var_idx, idx, :], origin='lower', aspect='auto',
                   extent=[X.min(), X.max(), Y.min(), Y.max()], cmap=cmap, norm=norm)
    dfs['b'][f'F_H_comp={H:.1f}'] = X.flatten()
    dfs['b'][f'M_H_comp={H:.1f}'] = Y.flatten()
    dfs['b'][f'Power_H_comp={H:.1f}'] = Xfm[var_idx, idx, :].flatten()
    col = [[1,1,1], [0,.75,1]]
    for k in range(2):
        jdx = np.argsort(ym[idx])
        xdata = peaks[idx[jdx],k]
        ydata = ym[idx[jdx]]
        func = lambda x,a,b: a*x**b
        popt,pcov = curve_fit(func, ydata, xdata)
        ax[i+1].plot(func(ydata, *popt), ydata, '--', color=col[k], lw=1)

    dx = np.log(F[-1] - cutoff) / 40
    dy = np.diff(ylim)[0] / 10
    xy = np.array([
        [Fpeak[var_idx,j], ylim[0] + dy],
        [np.exp(np.log(Fpeak[var_idx,j]) - dx), ylim[0]],
        [np.exp(np.log(Fpeak[var_idx,j]) + dx), ylim[0]]
    ])
    triangle = Polygon(xy, fc='w', ec='w')
    ax[i+1].add_patch(triangle)

    ax[i+1].set_xlim([cutoff, F[-1]])
    ax[i+1].set_xscale('log')
    ax[i+1].set_xlim(xticks[[0,-1]] + np.array([0,1]))
    ax[i+1].xaxis.set_major_locator(FixedLocator(xticks))
    ax[i+1].xaxis.set_minor_locator(NullLocator())
    if i == rows-1:
        ax[i+1].xaxis.set_major_formatter(FixedFormatter([f'{tick:g}' for tick in xticks]))
    else:
        ax[i+1].xaxis.set_major_formatter(FixedFormatter([]))

    yticks = np.linspace(ylim[0], ylim[1], 3)
    ax[i+1].text(xticks[-1]-3, ylim[1] - 0.1*np.diff(ylim), 'H = {:.1f} s'.format(H),
               fontsize=fontsize, color='w', verticalalignment='top', horizontalalignment='right')
    ax[i+1].set_ylim(ylim)
    ax[i+1].yaxis.set_major_locator(FixedLocator(yticks))
    ax[i+1].yaxis.set_major_formatter(FixedFormatter([f'{tick:.2f}' for tick in yticks]))
    ax[i+1].set_ylabel(r'M [GW$\cdot$s$^2$]')

cax = plt.axes([1 - x_offset[1] + 0.02, y_offset[0] + height[1] + y_space*1.7, w/20, h])
mappable = matplotlib.cm.ScalarMappable(norm=norm, cmap=cmap)
cbar = fig.colorbar(mappable, cax=cax, label='Power [dB]')
cbar.ax.set_yticks(_inverse(cbar_ticks))
cbar.ax.set_yticklabels([f'{tick:.0f}' for tick in cbar_ticks])

ax[-2].set_xlabel('Frequency [Hz]')

ax[-1].plot(H_comp[1:], Fpeak[var_idx,1:], 'ko-', lw=1, markersize=4, markerfacecolor='w')
dfs['c'] = pd.DataFrame(data={'Compensator_H': H_comp[1:], 'Frequency_peak': Fpeak[var_idx,1:]})
ax[-1].set_xlim([0.5, 6.5])
ax[-1].set_xticks(np.r_[1:7])
ax[-1].set_ylim([3, 16])
ax[-1].set_yticks(np.r_[5 : 20 : 5])
ax[-1].set_xlabel('Compensator inertia [s]')
ax[-1].set_ylabel('Frequency peak [Hz]')
ax[-1].grid(which='major', axis='y', ls=':', lw=0.5, color=[.6,.6,.6])
for side in 'right','top':
    for a in ax:
        a.spines[side].set_visible(False)

trans = mtransforms.ScaledTranslation(-0.5, 0, fig.dpi_scale_trans)
ax[0].text(0.0, 1.0, 'a', transform=ax[0].transAxes + trans, fontsize=fontsize, fontweight='bold', va='bottom')
ax[-1].text(0.0, 1.0, 'c', transform=ax[-1].transAxes + trans, fontsize=fontsize, fontweight='bold', va='bottom')
trans = mtransforms.ScaledTranslation(-0.5, 0.03, fig.dpi_scale_trans)
ax[1].text(0.0, 1.0, 'b', transform=ax[1].transAxes + trans, fontsize=fontsize, fontweight='bold', va='bottom')

fig.savefig(f'spectra_compensators_{var_names[var_idx]}.pdf')
save_dfs_dict(dfs, 'figure_8.xlsx')

### Figure 9

In [ ]:
# full grid without varying compensator's inertia
experiment_ID = 'f64bde90cab54d1ea770bb21f33c3ed1'
# full grid with variable compensator's inertia
experiment_ID = 'a40658acee3c4e419c0ee34d0c59f4df'
experiments_path = '../experiments/neural_network/'
Pfrac = 0.1
shelf_name = os.path.join(experiments_path,
                          experiment_ID,
                          experiment_ID[:6] + f'_Pfrac={Pfrac:.1f}.out')
db = shelve.open(shelf_name, flag='r')
print('The shelf `{}` contains the following experiments:'.format(os.path.basename(shelf_name)))
for i,expt in enumerate(db['experiments']):
    print('[{}] > {}'.format(i, expt['description']))

In [ ]:
CTRL_expt = db['experiments'][0]
expt = db['experiments'][7]
descr = expt['description']

pdf_file = 'prediction'
if descr == 'momentum steps varying only generators inertia, G3 type=2':
    pdf_file += '_G3_type=2'
elif descr == 'momentum steps varying only generators inertia, split G3 type=2':
    pdf_file += '_split_G3_type=2_Pfrac={:.1f}'.format(Pfrac)
elif descr == 'momentum steps varying only generators inertia, split G3 type=6':
    pdf_file += '_split_G3_type=6_Pfrac={:.1f}'.format(Pfrac)
elif descr == 'momentum steps varying inertia of G2 and VSG3, a VSG that replaces G3':
    pdf_file += '_VSG'
elif descr == 'momentum steps varying inertia of G2, G3 and VSG3':
    pdf_file += '_split_VSG_Pfrac={:.1f}'.format(Pfrac)
else:
    raise Exception('Unknown condition to consider')

var_name = 'Vd_bus3'
pdf_file += f'_{var_name}_{experiment_ID[:6]}.pdf'
time = expt['data_time']
dt = np.diff(time[:2])[0]
X = expt['data'][var_name]

time_wo_VSG = CTRL_expt['data_time']
X_wo_VSG = CTRL_expt['data'][var_name]

prediction_time = expt['prediction_time']
prediction = np.squeeze(expt['prediction'])
exact = expt['exact']
mean_prediction = expt['mean_prediction']
N_blocks = len(expt['H_values'])
block_dur = np.ceil(expt['data_time'][-1]) / N_blocks
area_measure = 'M'
measure_units = r'GW$\cdot$s$^2$'

offset = np.array([[0.175, 0.1], [0.01, 0.04]])
yspace_lower, yspace = 0.115, 0.025
h_lower = 0.25
w = 1 - np.sum(offset[:,0])
h = (1 - np.sum(offset[:,1]) - yspace_lower - yspace*2 - h_lower) / 3

fig = plt.figure(figsize=(COL_WIDTH[1], 4))
ax = {
    'a': plt.axes([offset[0,0], offset[0,1] + h_lower + yspace_lower + 2*(h + yspace), w, h]),
    'b': plt.axes([offset[0,0], offset[0,1] + h_lower + yspace_lower + h + yspace, w, h]),
    'c': plt.axes([offset[0,0], offset[0,1] + h_lower + yspace_lower, w, h]),
    'd': plt.axes([offset[0,0], offset[0,1], w, h_lower]),
}

dfs = make_dfs_dict('ab')
dur = 60
N_samples = int(dur / dt)

lbl = 'abc'
xticks = np.array([0.3, 0.5, 1, 2, 5, 10])

def plot_panel(time, t0, t1, X, N_samples, axx, color):
    idx, = np.where((time >= t0) & (time < t1))
    t = time[idx]
    N_trials = t.size // N_samples
    Y = np.reshape(X[idx][:N_trials*N_samples], (N_trials,-1))
    Yf = fft(Y)
    Yf = 2.0 / N_samples * np.abs(Yf[:, :N_samples//2])
    F = fftfreq(N_samples, dt)[:N_samples//2]
    Ydb = 20*np.log10(Yf.mean(axis=0))
    axx.semilogx(F, Ydb, color, lw=1)
    if color == 'k':
        jdx, = np.where((F>0.5) & (F<2))
        peaks,_ = find_peaks(Ydb[jdx], height=10, distance=10, prominence=3)
        dx = np.log(F[-1]) / 100
        dy = np.diff(axx.get_ylim())[0] / 50
        offset = 2
        for peak,fc in zip(peaks[:3], 'kkw'):
            Fpeak,Ydbpeak = F[jdx[peak]], Ydb[jdx[peak]]
            xy = np.array([
                [Fpeak, Ydbpeak + offset],
                [np.exp(np.log(Fpeak) - dx), Ydbpeak + offset + dy],
                [np.exp(np.log(Fpeak) + dx), Ydbpeak + offset + dy]
            ])
            triangle = Polygon(xy, fc=fc, ec='k', lw=0.75)
            axx.add_patch(triangle)
            axx.text(Fpeak, Ydbpeak+offset+dy, f'{Fpeak:.2f} Hz',
                     fontsize=fontsize-2, ha='center', va='bottom')
    return F,Ydb
    
for k in range(N_blocks):
    t0,t1 = block_dur*k, block_dur*(k+1)
    
    F,Ydb = plot_panel(time_wo_VSG, t0, t1, X_wo_VSG, N_samples, ax[lbl[k]], 'r')
    if k == 0: dfs['a']['Frequency'] = F
    dfs['a'][f'Power_without_VSG_M={exact[k]:.4f}'] = Ydb
    _,Ydb = plot_panel(time, t0, t1, X, N_samples, ax[lbl[k]], 'k')
    dfs['a'][f'Power_with_VSG_M={exact[k]:.4f}'] = Ydb
    ax[lbl[k]].text(10, 22.5, r'M = {:.4f} GW$\cdot$s$^2$'.format(expt['exact'][k]),
                    fontsize=fontsize, va='center', ha='right')
    
    idx, = np.where((prediction_time >= t0) & (prediction_time < t1) & np.logical_not(np.isnan(prediction)))
    n,x = np.histogram(prediction[idx], bins=10, density=True)
    ax['d'].plot(np.array([t0, t1])/60, mean_prediction[k] + np.zeros(2), color='k', lw=1)
    ax['d'].plot(np.array([t0, t1])/60, exact[k] + np.zeros(2), '--', color='m', lw=1)

dfs['b'] = pd.DataFrame(data={'Time': prediction_time/60, 'Predicted_M': prediction})
ax['d'].plot(prediction_time/60, prediction, color='k', lw=0.75)
ax['d'].set_xticks(np.r_[0 : 181 : 30])
ax['d'].set_xlabel('Time [min]')
ax['d'].set_ylabel(r'M [GW$\cdot$s$^2$]')
ax['d'].grid(which='major', axis='y', lw=0.5, ls=':', color=[.6,.6,.6])

for lbl in 'abcd':
    for side in 'right','top':
        ax[lbl].spines[side].set_visible(False)
    if lbl != 'd':
        ax[lbl].xaxis.set_minor_locator(NullLocator())
        ax[lbl].set_xlim(xticks[[0,-1]] + np.array([0,1]))
        ax[lbl].set_ylim([-16,31])
        ax[lbl].set_yticks(np.r_[-15 : 31 : 15])
        if lbl == 'c':
            ax[lbl].xaxis.set_major_locator(FixedLocator(xticks))
            ax[lbl].xaxis.set_major_formatter(FixedFormatter([f'{tick:g}' for tick in xticks]))
            ax[lbl].set_xlabel('Frequency [Hz]')
        else:
            ax[lbl].set_xticks([])
            ax[lbl].spines['bottom'].set_visible(False)
        ax[lbl].grid(which='major', axis='y', lw=0.5, ls=':', color=[.6,.6,.6])
ax['b'].set_ylabel('Power [dB]')

trans = mtransforms.ScaledTranslation(-0.5, 0, fig.dpi_scale_trans)
ax['a'].text(0.0, 1.0, 'a', transform=ax['a'].transAxes + trans, fontsize=fontsize,
             fontweight='bold', va='bottom')
ax['d'].text(0.0, 1.0, 'b', transform=ax['d'].transAxes + trans, fontsize=fontsize,
             fontweight='bold', va='bottom')

fig.savefig(pdf_file)
save_dfs_dict(dfs, 'figure_9.xlsx')

In [ ]:
db.close()

## Figure 10
Effect of varying the damping coefficient on the estimation of momentum.

In [ ]:
# full grid with variable compensator's inertia
experiment_ID = 'a40658acee3c4e419c0ee34d0c59f4df'
experiments_path = '../experiments/neural_network'
network_pars = pickle.load(open(os.path.join(experiments_path, experiment_ID, 'parameters.pkl'), 'rb'))
area_ID = network_pars['area_IDs'][0]
data_dir = network_pars['data_dirs'][0].format(area_ID)

damping_data_file = 'var_D_area_1.npz'
force = False
if force or not os.path.isfile(damping_data_file):
    data_files = sorted(glob.glob(os.path.join('..', data_dir, 'ieee39_*_D*_*.h5')))
    D = np.array([float(re.findall('D=\d.\d', d)[0].split('=')[1]) for d in data_files])
    window_dur = 60,
    window_step = window_dur
    var_names = network_pars['var_names']
    x_train_mean = network_pars['x_train_mean']
    x_train_std = network_pars['x_train_std']
    data_mean = {var_name: x_train_mean[k] for k,var_name in enumerate(var_names)}
    data_std = {var_name: x_train_std[k] for k,var_name in enumerate(var_names)}
    Xf = []
    for data_file in data_files:
        t, _, _, X_slide, _ = load_data_slide([data_file], var_names, data_mean, data_std,
                                                   window_dur, window_step, add_omega_ref=False,
                                                   verbose=True)
        dt = np.diff(t[:2])[0]
        Xf.append({})
        for var_name in var_names:
            N_samples = X_slide[var_name].shape[1]
            tmp = fft(X_slide[var_name])
            tmp = 2.0 / N_samples * np.abs(tmp[:, :N_samples//2])
            Xf[-1][var_name] = tmp.mean(axis=0)
            F = fftfreq(N_samples, dt)[:N_samples//2]
    data = {'F': F, 'Xf': Xf, 'd': D, 'window_dur': window_dur, 'window_step': window_step,
            'var_names': var_names, 'x_train_mean': x_train_mean, 'x_train_std': x_train_std}
    np.savez_compressed(damping_data_file, **data)
else:
    data = np.load(damping_data_file, allow_pickle=True)
    D, F, Xf = data['D'][:-1], data['F'], data['Xf'][:-1]
    var_names = data['var_names']

db = shelve.open(os.path.join(experiments_path, experiment_ID,
                              experiment_ID[:6]+'_Pfrac=0.1_KAD=2.0_KW=10.0.out'))
experiments = db['experiments'][9:9+len(D)]
db.close()

In [ ]:
dfs = []
for expt in experiments:
    damping = float(re.findall('\d.\d', expt['description'])[-1])
    M = expt['exact'][0]
    M_pred = np.squeeze(expt['prediction'])
    idx = np.where(np.logical_not(np.isnan(M_pred)))[0][0]
    df = pd.DataFrame(data={'D': damping, 'M': M, 'M_pred': M_pred[idx::60]})
    dfs.append(df)
df = pd.concat(dfs)
M = df.M.unique()[0]
df['MAPE'] = (df['M'] - df['M_pred']).abs() / df['M'] * 100
df_mean = df.groupby('D').mean()
df_mean

In [ ]:
palette = [[.8,.8,.8] for i in range(len(D))]
offset = np.array([[0.15,0.1], [0.12,0.05]])
space = [0.1, 0.1]
w = 1 - np.sum(offset[:,0])
h = 0.26
h_inset = 1 - np.sum(offset[:,1]) - 2*h - 2*space[1]
w_inset = (1 - np.sum(offset[:,0]) - space[0]) / 2
fig = plt.figure(figsize=(COL_WIDTH[1], 4))
ax = [
    plt.axes([offset[0,0], 1-offset[1,1]-h, w, h]),
    plt.axes([offset[0,0], offset[0,1], w, h]),
    plt.axes([offset[0,0], offset[0,1] + h + space[1]/1.4, w_inset, h_inset]),
    plt.axes([offset[0,0]+w_inset+space[0], offset[0,1] + h + space[1]/1.4, w_inset, h_inset])
]

dfs = make_dfs_dict('ab')
xlim = [-0.5, len(D)-0.5]
ax[0].plot(xlim, [M, M], 'r', lw=2, zorder=-1)
sns.violinplot(x='D', y='M_pred', data=df, cut=0, inner='quartile',
               palette=palette, ax=ax[0], linewidth=1, zorder=0)
dfs['a'] = df
twin = ax[0].twinx()
twin.plot(df_mean['MAPE'], 'ko', ms=7, markerfacecolor='w', markeredgewidth=1.5)
ax[0].set_xlim(xlim)
ax[0].set_xticklabels(range(len(D)))
ax[0].set_yticks(np.r_[0.2 : 0.25 : 0.01])
ax[0].set_xlabel('Damping')
ax[0].set_ylabel(r'M [GW$\cdot$s$^2$]')
ax[0].set_ylim([0.2, 0.24])
ax[0].set_yticks(np.linspace(0.2, 0.24, 5))
twin.set_ylim([1, 4.2])
twin.set_yticks(np.linspace(1, 4.2, 5))
twin.set_ylabel('MAPE [%]')

xlim = [[1e-2,20], [1e-2,1e-1], [0.4,1.5]]
ylim = [[-60,0], [-20,0], [-30,-10]]
var_name = 'Vd_bus3'
cmap = plt.get_cmap('viridis', len(Xf))
dfs['b']['F'] = F
for d,xf in zip(D, Xf):
    dfs['b'][f'Power_D={d:.1f}'] = 20*np.log10(xf[var_name])
for i,a in enumerate(ax[1:]):
    for k,xf in enumerate(Xf):
        a.plot(F, 20*np.log10(xf[var_name]), color=cmap(k), lw=1, label=f'D={k:.1f}')
    a.set_xscale('log')
    a.set_xlim(xlim[i])
    a.set_ylim(ylim[i])
    a.set_yticks(np.r_[ylim[i][0] : ylim[i][1]+5 : 20 if i == 0 else 10])
    a.grid(which='major', axis='x', lw=0.5, ls=':', color=[.6,.6,.6])
ax[1].set_xlabel('Frequency [Hz]')
ax[1].set_ylabel('Power [dB]')
ax[1].legend(loc='lower left', bbox_to_anchor=[-0.02, -0.08], frameon=False)
xticks = np.array([0.01, 0.03, 0.1, 0.3, 1, 3, 10, 20])
ax[1].xaxis.set_major_locator(FixedLocator(xticks))
ax[1].xaxis.set_minor_locator(NullLocator())
ax[1].xaxis.set_major_formatter(FixedFormatter([f'{tick:g}' for tick in xticks]))
ax[2].set_ylabel('Power [dB]')
ax[2].xaxis.set_major_locator(FixedLocator(xticks[:3]))
ax[2].xaxis.set_minor_locator(NullLocator())
ax[2].xaxis.set_major_formatter(FixedFormatter([f'{tick:g}' for tick in xticks[:3]]))
xticks = np.logspace(np.log10(xlim[2][0]), np.log10(xlim[2][1]), 4)
ax[3].xaxis.set_major_locator(FixedLocator(xticks))
ax[3].xaxis.set_minor_locator(NullLocator())
ax[3].xaxis.set_major_formatter(FixedFormatter([f'{tick:.1f}' for tick in xticks]))

for side in 'right','top':
    for a in ax[1:]:
        a.spines[side].set_visible(False)
        
trans = mtransforms.ScaledTranslation(-0.45, 0, fig.dpi_scale_trans)
ax[0].text(0.0, 1.0, 'a', transform=ax[0].transAxes+trans, fontsize=fontsize,
           fontweight='bold', va='bottom')
ax[2].text(0.0, 1.0, 'b', transform=ax[2].transAxes+trans, fontsize=fontsize,
           fontweight='bold', va='bottom')

fig.savefig('var_D_area_1.pdf')
save_dfs_dict(dfs, 'figure_10.xlsx')